# 1) Data engineering

In [ ]:
!pip install -q chromadb sentence-transformers mistralai gradio recipe-scrapers

In [ ]:
# --- IMPORTS  ---
import os
import json
import uuid
import time
import random
from google.colab import files

import requests
from bs4 import BeautifulSoup
from recipe_scrapers import scrape_me
import pandas as pd
from tqdm import tqdm
from sentence_transformers import SentenceTransformer

import chromadb
from mistralai import Mistral

import gradio as gr

In [ ]:
import getpass
api_key = getpass.getpass("Entrez votre clé API Mistral :")

In [ ]:
nb_pages = 42  # Nombre de pages Marmiton à parcourir
muffin_dataset = []

print("Début de l'extraction...")

# On parcourt les 42 pages de la recherche "muffin" sur Marmiton
for page in range(1, nb_pages + 1):
    print(f"Analyse de la page {page}...")
    search_url = f"https://www.marmiton.org/recettes/recherche.aspx?aqt=muffin&page={page}"

    try:
        # Utilisation d'un User-Agent pour simuler un navigateur et éviter un blocage
        response = requests.get(search_url, headers={'User-Agent': 'Mozilla/5.0'})
        soup = BeautifulSoup(response.content, 'html.parser')
        links = soup.select("a[href^='/recettes/recette_']")

        # Nettoyage des URLs de la page actuelle
        page_urls = list(set(["https://www.marmiton.org" + link['href'] for link in links]))

        for url in page_urls:
            try:
                scraper = scrape_me(url)
                # On ne filtre que les recettes dont le titre contient "muffin"
                if "muffin" in scraper.title().lower():
                  ing_list = scraper.ingredients()
                  ing_string = ", ".join(ing_list)
                  # Structuration des données pour l'indexation future dans ChromaDB
                  muffin_dataset.append({
                        "title": scraper.title(),
                        "ingredients": ing_string,
                        "instructions": scraper.instructions(),
                        "time": scraper.total_time(), # en minutes
                        "yield": scraper.yields(), # ex: "12 portions"
                        "url": url,
                        "page" : page
                    })
                # Pause d'une seconde pour respecter le serveur
                time.sleep(1)
            except Exception:
                continue

    except Exception as e:
        print(f"Erreur page {page}: {e}")

# Sauvegarde dans l'espace local de Colab
with open('muffins_marmiton_VF.json', 'w', encoding='utf-8') as f:
    json.dump(muffin_dataset, f, ensure_ascii=False, indent=4)

print(f"\n {len(muffin_dataset)} recettes sauvegardées dans muffins_marmiton_VF.json")

# Téléchargement automatique
files.download('muffins_marmiton_VF.json')

# 2, 3) Embedding - Vector store

In [ ]:
def load_and_prepare_data(json_path):
    """
    Charge le dataset brut et le transforme en dataframe
    """
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    df = pd.DataFrame(data)
    return df

In [ ]:
EMBEDDING_MODEL_NAME = "paraphrase-multilingual-MiniLM-L12-v2"
COLLECTION_NAME = "royaume_du_muffin"

def create_embeddings_and_store(df):
    """
    Transforme le texte en embeddings et les stocke dans ChromaDB.
    """
    print("🤖 Chargement du modèle d'embedding...")
    model = SentenceTransformer(EMBEDDING_MODEL_NAME)

    # Concaténation Titre + Ingrédients pour la recherche
    documents = (df['title'] + " : " + df['ingredients']).tolist()
    # Métadonnées : On conserve l'intégralité du DF
    metadatas = df.to_dict(orient='records')
    ids = [str(uuid.uuid4()) for _ in range(len(df))]

    print("⚡ Vectorisation en cours...")
    embeddings = model.encode(documents).tolist()

    # Stockage ChromaDB
    client = chromadb.Client() # En mémoire pour le test
    try: client.delete_collection(name=COLLECTION_NAME)
    except: pass

    # Création de la collection et indexation des vecteurs
    collection = client.create_collection(name=COLLECTION_NAME)
    collection.add(documents=documents, embeddings=embeddings, metadatas=metadatas, ids=ids)

    print(f"✅ Indexation terminée ! {collection.count()} recettes stockées.")
    return collection

In [ ]:
# 1. Chargement des données des recettes brutes
df = load_and_prepare_data('muffins_marmiton_VF.json')

# 2. On crée la base 'db' qui servira au Retrieval (RAG)
db = create_embeddings_and_store(df)

# 3. Instance locale du modèle pour les futures recherches utilisateurs
model = SentenceTransformer(EMBEDDING_MODEL_NAME)

# 4, 5) Retrieval - Generation

In [ ]:
model_mistral = "mistral-small-latest"

def interroger_chef_muffin(query_str):
    # Question de l'utilisateur vectorisée
    query_vector = model.encode([query_str]).tolist()

    # 1. RETRIEVAL : On cherche les 5 meilleures recettes dans la base ChromaDB
    search_results = db.query(query_embeddings=query_vector, n_results=5)
    recettes = search_results['metadatas'][0]

    # Podium affiché pour mes tests en interne, pas pour l'utilisateur externe
    podium_html = "### Podium des Recettes (Sources)\n"
    context_str = ""
    for i, r in enumerate(recettes):
        infos = f"**{i+1}. {r['title']}**\n- 🔗 [Lien vers la recette]({r['url']})\n\n"
        podium_html += infos
        # On garde le format complet pour le contexte envoyé au LLM
        context_str += f"--- {r['title']} ---\n{r['time']}\n{r['yield']}\n{r['ingredients']}\n{r['instructions']}\n{r['url']}\n"

    # 2. GENERATION
    prompt_final = f"""
    TU ES "CHEF MUFFIN". TON UNIQUE MISSION EST DE DONNER DES RECETTES COMPLÈTES DE MUFFINS, RIEN D'AUTRE. Réponds toujours avec humour et gentillesse : tu es un jeune chef drôle, avec un côté Reggaeman !

    ### RÈGLE D'OR :
    NE DEMANDE JAMAIS à l'utilisateur s'il veut les détails. DONNE-LES TOUT DE SUITE.
    L'utilisateur ne peut pas te répondre, c'est ta SEULE chance de l'aider !

    ### DIRECTIVES STRICTES :
    1. SÉLECTION : Soit l'utilisateur demande des ingrédients qui correspondent très bien à une recette du [CONTEXTE] en particulier, dans ce cas ne propose QUE CELLE-CI.
    Soit les recettes du [CONTEXTE] sont similaires mais ne correspondent pas parfaitement à la demande, propose les TOUTES, SEULEMENT SI ELLES CONTIENNENT AU MOINS UN INGREDIENT DE LA DEMANDE, ou si un ingrédient est de la bonne famille d'aliments .
    Si la [QUESTION] contient des objets non comestibles ou des ingrédients farfelus, ne cherche pas à les inclure. Réponds avec humour que Chef Muffin ne cuisine pas de [OBJET] et propose tes meilleures recettes sucrées à la place.

    2. FORMAT COMPLET : Pour chaque recette choisie, tu DOIS utiliser ce format précis :
       - ### [Emoji] [Titre exact]
       - **⏱ Durée :** [Durée] | **🧁 Portions :** [Nombre]

       **Ingrédients :**
       [Liste des ingrédients avec des tirets]

       **Instructions :**
       [Liste numérotée des étapes]

       🌐 Source : [URL]
    3. ANCRAGE : Si l'ingrédient précis n'existe pas dans le contexte, dis honnêtement que tu ne l'as pas en stock et DONNE LA RECETTE ENTIÈRE immédiatement après.
    4. PAS DE BLA-BLA : Évite les listes de titres inutiles. Si tu cites une recette, tu donnes ses instructions.
    5. LANGUE : Réponds toujours en français courant et appétissant. Ne fais pas que donner la recette, tu peux la présenter, dire si elle correspond aux ingrédients demandés ou pas.
    6. Utilise UNIQUEMENT les recettes fournies dans le bloc [CONTEXTE]. N'invente rien et NE MODIFIE JAMAIS une recette.


    [CONTEXTE]
    {context_str}

    [QUESTION]
    {query_str}
    """

    # 3. APPEL À MISTRAL
    client = Mistral(api_key=api_key)

    response = client.chat.complete(
        model=model_mistral,
        messages=[{"role": "user", "content": prompt_final}]
    )

    reponse_llm = response.choices[0].message.content

    return reponse_llm


In [ ]:
demo = gr.Interface(
    fn=interroger_chef_muffin,
    inputs=gr.Textbox(label="🌿 Qu'est ce que tu veux dans ton muffin, man ?"),
    outputs=gr.Markdown(label="👨‍🍳 La parole du Chef Winston"),
    title="Bienvenue dans l'atelier du Chef Winston Muffin ! 🇯🇲 ",
    description="Le Chef étoilé qui ne jure que par les muffins 🧁",
    theme="ocean",
    allow_flagging="never"
)

demo.launch(share=True)

# Alternative : on enrichit les embeddings en identifiant les identifiants principaux de chaque recette

Vous pouvez exécuter toutes ces cellules à la suite pour obtenir la version alternative.

In [ ]:
# --- CONFIGURATION ---
EMBEDDING_MODEL_NAME = "paraphrase-multilingual-MiniLM-L12-v2"
COLLECTION_NAME = "royaume_du_muffin"

client = Mistral(api_key=api_key)
model_embedding = SentenceTransformer(EMBEDDING_MODEL_NAME)

Fonction ***extraire_stars_avec_mistral_robuste*** utilisée pour le pré-traitement des données : identifie les ingrédients principaux grâce à Mistral pour améliorer la performance du RAG.

In [ ]:
def extraire_stars_avec_mistral_robuste(row, retries=5):
    prompt = f"""
    Analyse cette recette et extrais UNIQUEMENT les 2 ou 3 ingrédients signatures.
    Titre : {row['title']}
    Ingrédients : {row['ingredients']}
    Réponse (mots-clés séparés par une virgule uniquement) :"""

    for i in range(retries):
        try:
            response = client.chat.complete(
                model="mistral-tiny", # Modèle rapide pour l'extraction de mots-clés
                messages=[{"role": "user", "content": prompt}]
            )
            # Pause pour stabiliser le débit de requêtes
            time.sleep(0.5)
            # Nettoyage de la réponse pour garantir une uniformité dans la base de données
            return response.choices[0].message.content.strip().lower()

        except Exception as e:
            if "429" in str(e):
                # Stratégie de 'Backoff Exponentiel' : on attend
                # (2^i) + un facteur aléatoire pour éviter que toutes les requêtes ne repartent exactement au même moment
                attente = (2 ** i) + random.random()
                print(f" Trop rapide ! Pause de {attente:.1f}s...")
                time.sleep(attente)
            else:
                print(f"❌ Erreur sur '{row['title']}': {e}")
                return ""
    # Si après 5 tentatives rien ne fonctionne, on renvoie une chaîne vide pour ne pas bloquer le script
    return ""

Fonction ***enrichir_batch*** pour générer le dataset final "enrichi" : je l'ai fait par groupes de 100 recettes pour ne pas perdre les données déjà obtenues en cas de bug. Les ingrédients principaux sont répétés 3 fois dans l'embedding

In [ ]:
def enrichir_batch(df, batch_size=100, output_file="muffins_enriched_dataset.json"):
    df = df.copy()
    total_recettes = len(df)

    # On initialise la colonne 'ingrédients stars' (principaux) si elle n'existe pas
    if 'star_ingredients' not in df.columns:
        df['star_ingredients'] = ""

    print(f"Lancement du traitement de {total_recettes} muffins par blocs de {batch_size}...")

    for start_idx in range(0, total_recettes, batch_size):
        end_idx = min(start_idx + batch_size, total_recettes)
        print(f"\n Traitement du bloc : {start_idx} à {end_idx}...")

        # On ne traite que les lignes du bloc
        for idx in tqdm(range(start_idx, end_idx)):
            # On vérifie si on n'a pas déjà l'info (pour pouvoir relancer le script après un crash)
            if not df.loc[idx, 'star_ingredients']:
                df.loc[idx, 'star_ingredients'] = extraire_stars_avec_mistral_robuste(df.iloc[idx])

        # Sauvegarde intermédiaire après chaque bloc
        df.to_json(output_file, orient='records', force_ascii=False, indent=4)
        print(f"✅ Bloc terminé et sauvegardé dans {output_file}")

    # Une fois fini, on crée la colonne finale pour l'embedding
    df['text_for_embedding'] = df.apply(
        lambda x: f"{x['star_ingredients']} {x['star_ingredients']} {x['star_ingredients']} {x['title']} : {x['ingredients']}",
        axis=1
        )

    # Sauvegarde finale
    df.to_json(output_file, orient='records', force_ascii=False, indent=4)
    print(f"\n Traitement terminé ! Fichier sauvegardé : {output_file}")
    return df

Chargement du dataset optimisé comme pour la version standard


In [ ]:
def create_embeddings_and_store_optimized(df):
    print(f"🤖 Initialisation de ChromaDB...")

    # On vectorise la colonne 'text_for_embedding' (déjà préparée dans le JSON)
    documents = df['text_for_embedding'].astype(str).tolist()
    metadatas = df.to_dict(orient='records')
    ids = [str(uuid.uuid4()) for _ in range(len(df))]

    embeddings = model_embedding.encode(documents, show_progress_bar=True).tolist()

    client_db = chromadb.Client()
    try: client_db.delete_collection(name=COLLECTION_NAME)
    except: pass

    collection = client_db.create_collection(
        name=COLLECTION_NAME,
        metadata={"hnsw:space": "cosine"}
    )

    collection.add(documents=documents, embeddings=embeddings, metadatas=metadatas, ids=ids)
    print(f"✅ Base vectorielle prête : {collection.count()} muffins indexés.")
    return collection

In [ ]:
# On charge directement le fichier préparé
try:
    df_muffins = pd.read_json("muffins_enriched_dataset.json")
    print("✅ Grimoire chargé avec succès !")
except:
    print("❌ Erreur : le fichier 'muffins_enriched_dataset.json' est introuvable.")

# --- CRÉATION DE LA DB VECTORIELLE ---
db = create_embeddings_and_store_optimized(df_muffins)

In [ ]:
model_mistral = "mistral-small-latest"

def interroger_chef_muffin(query_str):
    # Question de l'utilisateur vectorisée
    query_vector = model_embedding.encode([query_str]).tolist()

    # 1. RETRIEVAL : On cherche les 5 meilleures recettes dans la base ChromaDB
    search_results = db.query(query_embeddings=query_vector, n_results=5)
    recettes = search_results['metadatas'][0]

    # Podium affiché pour mes tests en interne, pas pour l'utilisateur externe
    podium_html = "### Podium des Recettes (Sources)\n"
    context_str = ""
    for i, r in enumerate(recettes):
        infos = f"**{i+1}. {r['title']}**\n- 🔗 [Lien vers la recette]({r['url']})\n\n"
        podium_html += infos
        # On garde le format complet pour le contexte envoyé au LLM
        context_str += f"--- {r['title']} ---\n{r['time']}\n{r['yield']}\n{r['ingredients']}\n{r['instructions']}\n{r['url']}\n"

    # 2. GENERATION
    prompt_final = f"""
    TU ES "CHEF MUFFIN". TON UNIQUE MISSION EST DE DONNER DES RECETTES COMPLÈTES DE MUFFINS, RIEN D'AUTRE. Réponds toujours avec humour et gentillesse : tu es un jeune chef drôle, avec un côté Reggaeman !

    ### RÈGLE D'OR :
    NE DEMANDE JAMAIS à l'utilisateur s'il veut les détails. DONNE-LES TOUT DE SUITE.
    L'utilisateur ne peut pas te répondre, c'est ta SEULE chance de l'aider !

    ### DIRECTIVES STRICTES :
    1. SÉLECTION : Soit l'utilisateur demande des ingrédients qui correspondent très bien à une recette du [CONTEXTE] en particulier, dans ce cas ne propose QUE CELLE-CI.
    Soit les recettes du [CONTEXTE] sont similaires mais ne correspondent pas parfaitement à la demande, propose les TOUTES, SEULEMENT SI ELLES CONTIENNENT AU MOINS UN INGREDIENT DE LA DEMANDE, ou si un ingrédient est de la bonne famille d'aliments .
    Si la [QUESTION] contient des objets non comestibles ou des ingrédients farfelus, ne cherche pas à les inclure. Réponds avec humour que Chef Muffin ne cuisine pas de [OBJET] et propose tes meilleures recettes sucrées à la place.

    2. FORMAT COMPLET : Pour chaque recette choisie, tu DOIS utiliser ce format précis :
       - ### [Emoji] [Titre exact]
       - **⏱ Durée :** [Durée] | **🧁 Portions :** [Nombre]

       **Ingrédients :**
       [Liste des ingrédients avec des tirets]

       **Instructions :**
       [Liste numérotée des étapes]

       🌐 Source : [URL]
    3. ANCRAGE : Si l'ingrédient précis n'existe pas dans le contexte, dis honnêtement que tu ne l'as pas en stock et DONNE LA RECETTE ENTIÈRE immédiatement après.
    4. PAS DE BLA-BLA : Évite les listes de titres inutiles. Si tu cites une recette, tu donnes ses instructions.
    5. LANGUE : Réponds toujours en français courant et appétissant. Ne fais pas que donner la recette, tu peux la présenter, dire si elle correspond aux ingrédients demandés ou pas.
    6. Utilise UNIQUEMENT les recettes fournies dans le bloc [CONTEXTE]. N'invente rien et NE MODIFIE JAMAIS une recette.


    [CONTEXTE]
    {context_str}

    [QUESTION]
    {query_str}
    """

    # 3. APPEL À MISTRAL
    client = Mistral(api_key=api_key)

    response = client.chat.complete(
        model=model_mistral,
        messages=[{"role": "user", "content": prompt_final}]
    )

    reponse_llm = response.choices[0].message.content

    return reponse_llm


In [ ]:
demo = gr.Interface(
    fn=interroger_chef_muffin,
    inputs=gr.Textbox(label="🌿 Qu'est ce que tu veux dans ton muffin, man ? "),
    outputs=gr.Markdown(label="👨‍🍳 La parole du Chef Winston"),
    title="Bienvenue dans l'atelier du Chef Winston Muffin ! 🇯🇲 ",
    description="Le Chef étoilé qui ne jure que par les muffins 🧁",
    theme="ocean",
    allow_flagging="never"
)

demo.launch(share=True)